<a href="https://colab.research.google.com/github/chinmayeechintapanti/face_recognition/blob/main/face_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe opencv-python


In [ ]:
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📸 Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


In [ ]:
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
iris_indices = [468, 469, 470, 471, 472, 473, 474, 475]

def extract_face_embedding(image_path):
    img = cv2.imread(image_path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    with mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(rgb)
        if not results.multi_face_landmarks:
            return None
        landmarks = results.multi_face_landmarks[0]
        return np.array([[p.x, p.y, p.z] for p in landmarks.landmark]).flatten()

def extract_iris_embedding(image_path):
    img = cv2.imread(image_path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    with mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(rgb)
        if not results.multi_face_landmarks:
            return None
        landmarks = results.multi_face_landmarks[0]
        iris_data = []
        for i in iris_indices:
            p = landmarks.landmark[i]
            iris_data.extend([p.x, p.y, p.z])
        return np.array(iris_data)


In [ ]:
def register_user():
    os.makedirs("users", exist_ok=True)
    username = input("Enter a username: ")
    password = getpass.getpass("Enter a password: ")
    path = f'users/{username}.pkl'

    if os.path.exists(path):
        print("❌ User already exists.")
        return

    print("Look into the camera and click 'Capture'")
    photo_path = take_photo()

    face_embedding = extract_face_embedding(photo_path)
    iris_embedding = extract_iris_embedding(photo_path)

    if face_embedding is None or iris_embedding is None:
        print("❌ Could not detect face or iris.")
        return

    face_embedding = face_embedding / np.linalg.norm(face_embedding)
    iris_embedding = iris_embedding / np.linalg.norm(iris_embedding)

    user_data = {
        "username": username,
        "password": password,
        "face_embedding": face_embedding,
        "iris_embedding": iris_embedding
    }

    with open(path, 'wb') as f:
        pickle.dump(user_data, f)

    print(f"✅ User '{username}' registered successfully.")


In [ ]:
def login_user():
    print("Look into the camera and click 'Capture' to login.")
    photo_path = take_photo()

    face_embedding = extract_face_embedding(photo_path)
    iris_embedding = extract_iris_embedding(photo_path)

    if face_embedding is None or iris_embedding is None:
        print("❌ Could not detect face or iris.")
        return

    face_embedding = face_embedding / np.linalg.norm(face_embedding)
    iris_embedding = iris_embedding / np.linalg.norm(iris_embedding)

    best_match = None
    min_total_dist = float('inf')

    for file in os.listdir("users"):
        with open(f"users/{file}", "rb") as f:
            user_data = pickle.load(f)

        face_dist = np.linalg.norm(user_data["face_embedding"] - face_embedding)
        iris_dist = np.linalg.norm(user_data["iris_embedding"] - iris_embedding)
        total_dist = face_dist + iris_dist

        print(f"User: {user_data['username']} | Face: {face_dist:.4f} | Iris: {iris_dist:.4f}")

        if total_dist < min_total_dist:
            min_total_dist = total_dist
            best_match = user_data["username"]

    if min_total_dist < 2.0:  # You can adjust this threshold
        print(f"✅ Login successful. Welcome, {best_match}! (Score: {min_total_dist:.4f})")
    else:
        print("❌ Face + iris not recognized.")


In [ ]:
import os
import cv2
import pickle
import getpass
import numpy as np
import mediapipe as mp
from IPython.display import Javascript, display, Image
from google.colab.output import eval_js
from base64 import b64decode



In [ ]:
register_user()


Enter a username: abc
Enter a password: ··········
Look into the camera and click 'Capture'


<IPython.core.display.Javascript object>

✅ User 'abc' registered successfully.


In [ ]:
login_user()


Look into the camera and click 'Capture' to login.


<IPython.core.display.Javascript object>

User: abc | Face: 0.0067 | Iris: 0.0122
✅ Login successful. Welcome, abc! (Score: 0.0189)
